In [9]:
from matplotlib import pyplot as plt
from matplotlib.pyplot import imshow
import gym
from PIL import Image, ImageDraw
import numpy as np

%matplotlib inline

In [64]:
plt.rcParams['figure.figsize'] = [18, 12]
HEADER_HEIGHT = 34

In [65]:
def preprocess_atari_crop(I):
  I = I.copy()
#   I = I[35:195]  # crop
  I = I[HEADER_HEIGHT:194]
  I[I[:, :, 0] == 144, :] = 0  # erase background (background type 1)
  I[I[:, :, 0] == 109, :] = 0  # erase background (background type 2)
  I[157:, :, :] = 0
  return I

def preprocess_atari_object(image):
    I = preprocess_atari_crop(image)
    object_coord = np.zeros((6, ))

    for idx, target in enumerate([92, 236, 213]):
        coord_tuple = np.where(I[:, :, 0] == target)

        if len(coord_tuple[0]) > 0 and len(coord_tuple[1]) > 0:
#             object_coord[idx*2] = (np.mean(coord_tuple[1]))
#             object_coord[idx*2 + 1] = np.mean(coord_tuple[0])
            object_coord[idx*2] = (np.min(coord_tuple[1]))
            object_coord[idx*2 + 1] = np.min(coord_tuple[0])

            
    # Return the object coordinates
    return object_coord.reshape((1, 6, 1))


In [66]:
def annotate_img(arr):
    objvec = preprocess_atari_object(arr).reshape([-1])
    img = Image.fromarray(arr)
    x_a,y_a,x_b,y_b,x_p,y_p = objvec.astype(np.int)
    draw = ImageDraw.Draw(img)
    draw.rectangle([x_a,HEADER_HEIGHT+y_a,x_a,HEADER_HEIGHT+y_a], fill='red', outline=None)
    draw.rectangle([x_b,HEADER_HEIGHT+y_b,x_b,HEADER_HEIGHT+y_b], fill='black', outline=None)
    draw.rectangle([x_p,HEADER_HEIGHT+y_p,x_p,HEADER_HEIGHT+y_p], fill='blue', outline=None)
    return np.asarray(img.resize((img.width*10,img.height*10)))
#     imshow(np.asarray(img.resize((img.width*6,img.height*6))))
#     plt.imsave(fname='step_%d'%step, np.asarray(img.resize((img.width*6,img.height*6))))

In [67]:
import time
from IPython import display
env = gym.make('PongDeterministic-v4')
arr = env.reset()
NAME = 'min_step_%d.png'
plt.imsave(fname=NAME%0, arr=annotate_img(arr))
# plt.imshow(annotate_img(arr))
# display.clear_output(wait=True)
# display.display(plt.gcf())
# time.sleep(2)
for step in range(1, 200):
    arr = env.step(env.action_space.sample())[0]
    plt.imsave(fname=NAME%step, arr=annotate_img(arr))
    if step % 20 == 0:
        print('current step:', step)
#     plt.imshow(annotate_img(arr))
#     display.clear_output(wait=True)
#     display.display(plt.gcf())
#     time.sleep(0.5)

current step: 20
current step: 40
current step: 60
current step: 80
current step: 100
current step: 120
current step: 140
current step: 160
current step: 180
